In [ ]:
import pandas as pd
import os
import datetime
sop=pd.read_excel('C:\Users\liqing.cai\Desktop\sop.xlsx',sheetname='sop',header=None)
id= datetime.datetime.fromtimestamp(os.path.getmtime('C:\Users\liqing.cai\Desktop\statsSummary.xlsx')).strftime('%Y%m%d%H%M%S')
df=pd.DataFrame(sop)
sop_len=len(sop)
chemical_result=pd.DataFrame(columns=['material_name','description','batch_id'])
dna_result=pd.DataFrame(columns=['material_name','description','batch_id'])
wash_result=pd.DataFrame(columns=['material_name','description','batch_id'])
sop_result=pd.DataFrame()
for i in range(0,sop_len):
    d1=df.iloc[i][0]
    if d1=='chemical':
        chemical=pd.DataFrame(df.iloc[i,1:]).T
        chemical.columns=['material_name','description','batch_id']
        chemical_result=chemical_result.append(chemical)
    if d1=='dna':
        dna=pd.DataFrame(df.iloc[i,1:]).T
        dna.columns=['material_name','description','batch_id']
        dna_result=dna_result.append(dna)
    if d1=='wash':
        wash=pd.DataFrame(df.iloc[i,1:]).T
        wash.columns=['material_name','description','batch_id']
        wash_result=wash_result.append(wash)
    if d1=='Experiment Purpose':
        sop_result=sop_result.join(pd.DataFrame(df.iloc[i+1]).T,how='outer')
    if d1=='equipment':
        t=pd.DataFrame(df.iloc[i,1:]).T
        t.index=[1]
        sop_result=sop_result.join(t,how='outer',lsuffix='t')
    if d1=='method_step':
        t=df.iloc[i+1:,1:3]
        for a in range(0,len(t)):
            a=pd.DataFrame(t.iloc[a]).T
            a.index=[1]
            sop_result=sop_result.join(a,how='outer',lsuffix='a')
sop_result['id']=id

In [ ]:
import psycopg2
psql=psycopg2.connect(database='efirm',user='postgres',password='abc123...',host='172.16.0.70',port='5432')
cur=psql.cursor()
cur.execute("select * from sop_result") #接受sql语句，这里用key值作为标识每条记录的主键
id=cur.fetchall()
for i in range(0,len(sop_result)):
    # print sop_result.iloc[i].tolist()[0]
    cur.execute("insert into sop_result(purpose,result,date,operator,eplate_id,eplate_description,eplate_batch_id,efirm_id,efirm_description,efirm_batch_id,polymer,polymer_efirm,polymer_wash,sample,sample_efirm,sample_wash,detector,sample_incubation,detector_wash,reporter,reporter_efirm,reporter_wash,readout,readout_efirm,id) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) ",(sop_result.iloc[i].tolist()[0],sop_result.iloc[i].tolist()[1],sop_result.iloc[i].tolist()[2],sop_result.iloc[i].tolist()[3],sop_result.iloc[i].tolist()[4],sop_result.iloc[i].tolist()[5],sop_result.iloc[i].tolist()[6],sop_result.iloc[i].tolist()[7],sop_result.iloc[i].tolist()[8],sop_result.iloc[i].tolist()[9],sop_result.iloc[i].tolist()[10],sop_result.iloc[i].tolist()[11],sop_result.iloc[i].tolist()[12],sop_result.iloc[i].tolist()[13],sop_result.iloc[i].tolist()[14],sop_result.iloc[i].tolist()[15],sop_result.iloc[i].tolist()[16],sop_result.iloc[i].tolist()[17],sop_result.iloc[i].tolist()[18],sop_result.iloc[i].tolist()[19],sop_result.iloc[i].tolist()[20],sop_result.iloc[i].tolist()[21],sop_result.iloc[i].tolist()[22],sop_result.iloc[i].tolist()[23],sop_result.iloc[i].tolist()[24]))
    psql.commit()
cur.close()
psql.close()